In [1]:
%pip install simalign

In [2]:
!pip install transformers sentencepiece torch --quiet

In [3]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 62.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import json
import pandas as pd
from simalign import SentenceAligner
import spacy
from transformers import MarianMTModel, MarianTokenizer

In [5]:
# === Load SpaCy models ===
nlp_de = spacy.load("de_core_news_sm")  # German lemmatizer
nlp_en = spacy.load("en_core_web_sm")   # English lemmatizer

In [7]:
# === Load dictionary JSON (German → English mappings) ===
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/german_english.json"

with open(path, "r", encoding="utf-8") as f:
    vocab_dict = json.load(f)

Mounted at /content/drive


In [8]:
# === MarianMT Models ===
de_en_model = "Helsinki-NLP/opus-mt-de-en"
en_de_model = "Helsinki-NLP/opus-mt-en-de"

tokenizer_de_en = MarianTokenizer.from_pretrained(de_en_model)
translator_de_en = MarianMTModel.from_pretrained(de_en_model)

tokenizer_en_de = MarianTokenizer.from_pretrained(en_de_model)
translator_en_de = MarianMTModel.from_pretrained(en_de_model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [9]:
# === SimAlign ===
aligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

2025-11-08 16:12:00,331 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [10]:
# === Utility functions ===
def normalize_word(word, lang="de"):
    """Lemmatize word for dictionary lookup"""
    nlp = nlp_de if lang == "de" else nlp_en
    doc = nlp(word)
    return doc[0].lemma_

In [11]:
def get_literal_and_root(word, lang="de"):
    """Return lemma + dictionary meaning"""
    lemma = normalize_word(word, lang)
    if lang == "de":
        literal = vocab_dict.get(lemma, "N/A")  # German → English
    else:
        # reverse lookup for English → German
        reverse_dict = {v: k for k, v in vocab_dict.items()}
        literal = reverse_dict.get(lemma, "N/A")
    return lemma, literal

In [12]:
def translate_sentence(sentence, src_lang="de"):
    """Translate using MarianMT"""
    if src_lang == "de":
        inputs = tokenizer_de_en(sentence, return_tensors="pt", padding=True, truncation=True)
        translated = translator_de_en.generate(**inputs)
        return tokenizer_de_en.decode(translated[0], skip_special_tokens=True)
    else:
        inputs = tokenizer_en_de(sentence, return_tensors="pt", padding=True, truncation=True)
        translated = translator_en_de.generate(**inputs)
        return tokenizer_en_de.decode(translated[0], skip_special_tokens=True)

In [13]:
def build_vocab_chart(src_sentence_str, src_lang="de"):
    """Build vocab chart for either German→English or English→German"""
    nlp = nlp_de if src_lang == "de" else nlp_en
    src_sentence = [token.text for token in nlp(src_sentence_str)]

    trg_sentence_str = translate_sentence(src_sentence_str, src_lang=src_lang)
    trg_sentence = trg_sentence_str.split()

    alignments = aligner.get_word_aligns(src_sentence, trg_sentence)
    aligned_pairs = alignments["itermax"]

    vocab_chart = []
    seen = []
    for src_idx, trg_idx in aligned_pairs:
        src_word = src_sentence[src_idx]
        if not src_word.isalpha() or src_word in seen:
            continue

        context_meaning = trg_sentence[trg_idx]
        root_word, literal_meaning = get_literal_and_root(src_word, lang=src_lang)

        vocab_chart.append({
            f"{'German' if src_lang=='de' else 'English'} Word": src_word,
            "Context Meaning": context_meaning,
            "Root Word": root_word,
            "Literal Meaning": literal_meaning
        })
        seen.append(src_word)

    return src_sentence_str, trg_sentence_str, pd.DataFrame(vocab_chart)


In [14]:
# === Example German → English ===
src_de = "Frau Meier geht heute in den Supermarkt."
german, english, df_de_en = build_vocab_chart(src_de, src_lang="de")

print("=== DE → EN ===")
print("German:  ", german)
print("English: ", english)
print(df_de_en.to_string(index=False))

=== DE → EN ===
German:   Frau Meier geht heute in den Supermarkt.
English:  Mrs. Meier is going to the supermarket today.
German Word Context Meaning  Root Word Literal Meaning
       Frau            Mrs.       Frau    woman (irr.)
      Meier           Meier      Meier             N/A
       geht              is      gehen       go (irr.)
      heute          today.      heute           today
         in              to         in              in
        den             the        der             the
 Supermarkt     supermarket Supermarkt     supermarket


In [15]:
# === Example English → German ===
#src_en = "Mrs. Meier is going to the supermarket today."
english, german, df_en_de = build_vocab_chart(english, src_lang="en")

print("\n=== EN → DE ===")
print("English: ", english)
print("German:  ", german)
print(df_en_de.to_string(index=False))



=== EN → DE ===
English:  Mrs. Meier is going to the supermarket today.
German:   Mrs. Meier geht heute in den Supermarkt.
English Word Context Meaning   Root Word Literal Meaning
       Meier           Meier       meier             N/A
          is            geht          be             N/A
          to              in          to              zu
         the             den         the             die
 supermarket     Supermarkt. supermarket      Supermarkt
       today           heute       today           heute


## Morphology Chart

In [16]:
import spacy
import pandas as pd

# Load German spaCy model
nlp = spacy.load("de_core_news_sm")

In [17]:
# Expanded mapping dictionary for abbreviations → full forms
morph_map = {
    "Nom": "Nominative (Subject)",
    "Acc": "Accusative (Direct Object)",
    "Dat": "Dative (Indirect Object)",
    "Gen": "Genitive (Possessive)",
    "Sing": "Singular",
    "Plur": "Plural",
    "Masc": "Masculine",
    "Fem": "Feminine",
    "Neut": "Neuter",
    "1": "1st Person",
    "2": "2nd Person",
    "3": "3rd Person",
    "Ind": "Indicative",
    "Subj": "Subjunctive",
    "Imp": "Imperative",
    "Pres": "Present",
    "Past": "Past",
    "Fut": "Future",
    "Prs": "Personal Pronoun",
    "Rel": "Relative Pronoun",
    "Dem": "Demonstrative Pronoun",
    "Int": "Interrogative Pronoun",
    "Art": "Article",
    "Neg": "Negative Pronoun",
    "Fin": "Finite Verb",
    "Inf": "Infinitive",
    "Part": "Participle",
    "Sup": "Supine"
}

In [18]:
def expand_morphology(morph_dict):
    """Expand morphological abbreviations into full forms"""
    expanded = []
    for k, v in morph_dict.items():
        expanded.append(f"{k}: {morph_map.get(v, v)}")
    return ", ".join(expanded)

In [19]:
def pos_morphology_chart(sentence):
    """Generate POS + Morphology chart for a German sentence"""
    doc = nlp(sentence)
    rows = []
    for token in doc:
        if token.is_alpha:  # skip punctuation
            morph = token.morph.to_dict()
            expanded_morph = expand_morphology(morph)
            rows.append({
                "Word": token.text,
                "Lemma": token.lemma_,
                "POS": token.pos_,
                "Morphology": expanded_morph if expanded_morph else "—"
            })
    return pd.DataFrame(rows)


In [20]:
# Example

df = pos_morphology_chart(src_de)

print(df.to_string(index=False))

      Word      Lemma   POS                                                                                              Morphology
      Frau       Frau  NOUN                                          Case: Nominative (Subject), Gender: Feminine, Number: Singular
     Meier      Meier PROPN                                          Case: Nominative (Subject), Gender: Feminine, Number: Singular
      geht      gehen  VERB           Mood: Indicative, Number: Singular, Person: 3rd Person, Tense: Present, VerbForm: Finite Verb
     heute      heute   ADV                                                                                                       —
        in         in   ADP                                                                                                       —
       den        der   DET Case: Accusative (Direct Object), Definite: Def, Gender: Masculine, Number: Singular, PronType: Article
Supermarkt Supermarkt  NOUN                                   Case: Accusati

In [21]:
!pip install phonemizer epitran gTTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 38.3 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10744 sha256=739f7c3bd86b0eaafbf38c317db9b633232df9b8a92e24ecdfd0e065e58b34a0
  Stored in directory: /root/.cache/pip/wheels/f2/67/7d/2e80818c2f3dc8f0735d0810338c47e95d3212114ab97b4ede
Successfully built unicodecsv
  Attempting uninstall: cl

In [22]:
!pip install git+https://github.com/openai/whisper.git
!apt install ffmpeg -y


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-llpno4cx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-llpno4cx
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=eca651147f39dc2871585edb1c6692ca128b0a646be25882627156e30dcf2751
  Stored in directory: /tmp/pip-ephem-wheel-cache-d8j2x72s/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [23]:
from google.colab import files
uploaded = files.upload()


Saving Universität_wrong.m4a to Universität_wrong.m4a


In [25]:
import whisper

model = whisper.load_model("tiny")  # you can use "base" or "small" for higher accuracy
file_name = "Universität_wrong.m4a"  # change this to the uploaded file name

target_audio = "Universität"

result = model.transcribe(file_name, language="de")  # "en" for English
spoken_text = result["text"].strip()

print("🗣️ Recognized speech:", spoken_text)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣️ Recognized speech: University.


In [26]:
from phonemizer import phonemize
import epitran
from difflib import SequenceMatcher

epi_de = epitran.Epitran("deu-Latn")
epi_en = epitran.Epitran("eng-Latn")

In [27]:
def get_pronunciation(word, lang="de"):
    try:
        ipa = epi_de.transliterate(word) if lang == "de" else epi_en.transliterate(word)
        if not ipa.strip():
            ipa = phonemize(
                word,
                language="de" if lang == "de" else "en-us",
                backend="espeak",
                strip=True,
                preserve_punctuation=True
            )
        return ipa
    except Exception as e:
        return f"Error: {e}"

In [28]:
def compare_pronunciation(spoken_word, target_word, lang="de"):
    ipa_spoken = get_pronunciation(spoken_word, lang)
    ipa_target = get_pronunciation(target_word, lang)

    print(f"\n🎯 Target: {target_word} → {ipa_target}")
    print(f"🗣 You said: {spoken_word} → {ipa_spoken}")

    score = SequenceMatcher(None, ipa_spoken, ipa_target).ratio()
    print(f"✅ Pronunciation similarity: {score*100:.1f}%")

compare_pronunciation(spoken_text,target_audio, lang="de")



🎯 Target: Universität → uːniːfrziːtɛt
🗣 You said: University. → uːniːfrziːtyː.
✅ Pronunciation similarity: 81.5%


In [29]:
!apt update
!apt install -y espeak-ng


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,969 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/

In [30]:
from gtts import gTTS
from IPython.display import Audio

tts = gTTS(target_audio, lang="de")
tts.save("correct_pronunciation.mp3")
Audio("correct_pronunciation.mp3", autoplay=True)


In [31]:
from IPython.display import Audio
!espeak-ng -v de+f3 -s 100 -w output.wav "{target_audio}"
from IPython.display import Audio
Audio("output.wav", autoplay=True)


In [32]:
# === German Question Generation via Translation + FLAN-T5 ===

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

from transformers import MarianMTModel, MarianTokenizer

In [33]:
de_en_model = "Helsinki-NLP/opus-mt-de-en"
en_de_model = "Helsinki-NLP/opus-mt-en-de"

tokenizer_de_en = MarianTokenizer.from_pretrained(de_en_model)
translator_de_en = MarianMTModel.from_pretrained(de_en_model)

tokenizer_en_de = MarianTokenizer.from_pretrained(en_de_model)
translator_en_de = MarianMTModel.from_pretrained(en_de_model)

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [34]:
# FLAN-T5 (question generation in English)
qg_model_name = "google/flan-t5-base"
qg_tokenizer = AutoTokenizer.from_pretrained(qg_model_name)
qg_model = AutoModelForSeq2SeqLM.from_pretrained(qg_model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [35]:
# ---- Helper functions ----
def translate_de_to_en(text):
    inputs = tokenizer_de_en(text, return_tensors="pt", padding=True, truncation=True)
    outputs = translator_de_en.generate(**inputs)
    return tokenizer_de_en.decode(outputs[0], skip_special_tokens=True)

In [36]:
def translate_en_to_de(text):
    inputs = tokenizer_en_de(text, return_tensors="pt", padding=True, truncation=True)
    outputs = translator_en_de.generate(**inputs)
    return tokenizer_en_de.decode(outputs[0], skip_special_tokens=True)

In [37]:
def generate_questions_en(text, num_questions=3):
    prompt = f"Generate {num_questions} comprehension questions in English about the following text:\n{text}\nQuestions:"
    inputs = qg_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = qg_model.generate(
        **inputs,
        max_length=128,
        num_return_sequences=num_questions,
        do_sample=True,
        temperature=0.8,
        top_p=0.9
    )
    return [qg_tokenizer.decode(o, skip_special_tokens=True).strip() for o in outputs]

In [38]:
def generate_german_questions(paragraph, num_questions=3):
    # Translate German → English
    english_text = translate_de_to_en(paragraph)

    # Generate English questions
    english_questions = generate_questions_en(english_text, num_questions=num_questions)

    # Translate back to German
    german_questions = [translate_en_to_de(q) for q in english_questions]
    return list(zip(english_questions, german_questions))


In [40]:
paragraph = """
Heute ist der erste Schultag. Lena steht mit ihrer Schultüte vor der Schule. Sandra, Susanne und Paul sind auch da. Die Kinder kennen sich aus dem Kindergarten. Jetzt gehen sie in die gleiche Klasse. Sie freuen sich schon auf den Unterricht. Lena freut sich besonders auf das Rechnen. Sandra und Susanne aufs Schreiben. Und Paul? Paul sagt, er freut sich besonders auf die Pausen.
In der Klasse lernen sie ihren Lehrer, Herrn Mayer, kennen. Herr Mayer ist noch sehr jung und lustig. In der ersten Stunde lernen die Kinder das ABC-Lied. Alle singen begeistert mit.
Danach schreibt der Lehrer die ersten Buchstaben an die Tafel: A wie Affe, B wie Banane. Herr Mayer zeichnet einen Affen dazu, der einen Banane frisst. Die Kinder lachen laut. Dann läutet auch schon die Schulglocke.
Der erste Tag in der Schule ist vorbei. Vor der Schule warten die Eltern auf die Kinder. Die Kinder erzählen vom ersten Tag. Sie freuen sich schon auf morgen
"""

results = generate_german_questions(paragraph, num_questions=10)

print("*******Fragen:*******\n")
for i, (en_q, de_q) in enumerate(results, 1):
    print(f"{i}. 🇩🇪: {de_q}\n   ({en_q})\n")


*******Fragen:*******

1. 🇩🇪: Wie lange sind sie in der Klasse?
   (How long are they in the class?)

2. 🇩🇪: Wo sind Sandra, Susanne und Paul?
   (Where are Sandra, Susanne and Paul?)

3. 🇩🇪: Worauf freuen sich die Kinder?
   (What are the children looking forward to?)

4. 🇩🇪: Was hält Sandra von ihren Klassenkameraden?
   (What does Sandra think of her classmates?)

5. 🇩🇪: Worauf freuen sie sich?
   (What are they looking forward to?)

6. 🇩🇪: Worauf freut sich Sandra?
   (What is Sandra looking forward to?)

7. 🇩🇪: Wo treffen die Kinder ihren Lehrer?
   (Where do the children meet their teacher?)

8. 🇩🇪: Wo kommen die Kinder her?
   (Where are the children from?)

9. 🇩🇪: Wie heißen die vier Kinder?
   (What are the names of the four children?)

10. 🇩🇪: Was machen die Kinder jetzt?
   (What are the children doing now?)

